In [ ]:
import json
import os

def sample(file_path: str, file_name: str, limit = 100):
    
    sample_path = os.path.join(file_path, f'sample_{limit}')
    if os.path.exists(sample_path) == False:
        os.mkdir(sample_path)

    with open(os.path.join(file_path, file_name)) as file:
        sample_lines = [next(file) for x in range(limit)]

    with open(os.path.join(sample_path, file_name), 'a') as file_sample:
        file_sample.writelines(sample_lines)
    

In [ ]:
# sample('C:/Users/Krips/Documents/Programming/PDT/','conversations.jsonl', 100000)
# sample('C:/Users/Krips/Documents/Programming/PDT/','conversations.jsonl', 10000)
# sample('C:/Users/Krips/Documents/Programming/PDT/','conversations.jsonl', 1000)
# sample('C:/Users/Krips/Documents/Programming/PDT/','authors.jsonl', 100000)
# sample('C:/Users/Krips/Documents/Programming/PDT/','authors.jsonl', 10000)
# sample('C:/Users/Krips/Documents/Programming/PDT/','authors.jsonl', 1000)
sample('C:/Users/Krips/Documents/Programming/PDT/','authors.jsonl', 10)

In [ ]:
from datetime import datetime
import json


def row_traverse(file_path: str, file_name: str, limit=1_000_000):
    start_time = datetime.now()

    unique = {}
    duplicates = set()

    count = 0

    with open(os.path.join(file_path, file_name)) as file:
        for line in file:
            count += 1
            _json_file = json.loads(line)
            if _json_file.get("conversation_id", "None") in unique:
                duplicates.add(_json_file.get("conversation_id"))
            else:
                unique[_json_file.get("conversation_id")] = 1

            if count % 1_000_000 == 0:
                print(
                    "-"
                    * (
                        len(datetime.now().isoformat()) + 6
                        if len(datetime.now().isoformat()) > len(str(count))
                        else len(str(count))
                    )
                )
                print(f"time: {datetime.now().isoformat()}")
                print(count)
    end_time = datetime.now()

    print(f"time to complete: {end_time-start_time}")


In [ ]:
result_time = 0
for i in range(100):
    _time = row_traverse('C:/Users/Krips/Documents/Programming/PDT/sample_10000/','conversations.jsonl')
    result_time += _time
     
print(result_time/100)

In [55]:
class Extractor:
    
    @staticmethod
    def generate_author_row(_input:dict = {}):
        return {
            'id': _input.get('id'),
            'name': _input.get('name'),
            'username': _input.get('username'),
            'description': _input.get('description'),
            'followers_count': _input.get('public_metrics.followers_count'),
            'following_count': _input.get('public_metrics.following_count'),
            'tweet_count': _input.get('public_metrics.tweet_count'),
            'listed_count': _input.get('public_metrics.listed_count'),
        }

    @staticmethod
    def create_author_table():
        return ''' 
            CREATE TABLE authors (
                id bigint PRIMARY KEY,
                name VARCHAR ( 255 ),
                username VARCHAR ( 255 ),
                description text,
                followers_count integer,
                following_count integer,
                tweet_count integer,
                listed_count integer
            );
        '''

In [56]:
from datetime import datetime
import json
import time
from pathlib import Path
import csv
import asyncio
import threading, queue

def write_to_csv(writer, row):
    writer.writerow(row)


def author_row_traverse(file_path: str, file_name: str, block_max_size=10_000):
    unique = {}
    duplicates = set()
    
    block_size = 0

    import_start_time = datetime.now()
    block_start_time = datetime.now()
    with open(os.path.join(file_path, Path(file_name).with_suffix('.csv')), 'w', encoding="utf-8", newline='') as f:
        writer = csv.writer(f, escapechar='\\')
        writer.writerow(Extractor.generate_author_row().keys())

        with open(os.path.join(file_path, file_name), encoding="utf-8") as file:
            for line in file:
                block_size += 1
        
                _json_file = json.loads(line)        
                if _json_file.get("id", "None") in unique:
                    duplicates.add(_json_file.get("id"))
                else:
                    unique[_json_file.get("id")] = 1
                    writer.writerow(Extractor.generate_author_row(_json_file).values())
                    
                if block_size == block_max_size:
                    block_end_time = datetime.now()
                    _time_delta = (block_end_time-block_start_time)
                    print("-" * 50)
                    print(f"time now: {block_end_time.isoformat()}")
                    print(f"block excecution time: {int(_time_delta.seconds/60)}:{int(_time_delta.seconds % 60)}")
                    block_start_time = block_end_time
                    block_size = 0

    import_end_time = datetime.now()
    _time_delta = (import_end_time-import_start_time)
    print(f"import excecution time: {int(_time_delta.seconds/60)}:{int(_time_delta.seconds % 60)}")

In [ ]:
# author_row_traverse('C:/Users/Krips/Documents/Programming/PDT/sample_10/', 'authors.jsonl', block_max_size=100)
author_row_traverse('C:/Users/Krips/Documents/Programming/PDT/', 'authors.jsonl', block_max_size=10_000)

In [57]:
import psycopg2

conn = psycopg2.connect(
    dbname='PDT',
    user='postgres',
    password='291122',
    host='localhost',
    port='5432'
)
with conn.cursor() as cursor:
    cursor.execute(Extractor.create_author_table())
    conn.commit()
    # cursor.execute("SELECT * FROM authors")
    # print(cursor.fetchone())
    
    cursor.close()
    # commit the changes

connected
